# torch

In [1]:
import torch
import numpy as np

## 1. Tensors

### is_tensor

In [ ]:
x = torch.tensor([1, 2, 3])
print(torch.is_tensor(x))

y = np.array([1, 2, 3])
print(torch.is_tensor(y))

True
False


### is_floating_point

In [ ]:
x = torch.DoubleTensor([1, 2, 3])
y = torch.LongTensor([1, 2, 3])

print(torch.is_floating_point(x))
print(torch.is_floating_point(y))

True
False


### is_nonzero

Returns True if the input is a **single element tensor** which is not equal to zero after type conversions.


In [ ]:
x = torch.randn((1, 1))

print(torch.is_nonzero(x))

True


### numel

Returns **the total number of elements** in the input tensor.

In [ ]:
x = torch.randn((2, 2))

print(torch.numel(x))

4


## 2. Creation Ops ⭐️⭐️⭐️

### tensor
- torch.tensor(data, dtype, device, requires_grad, pin_memory)
- torch.tensor() always copies data -> 메모리 공유 X
    - 메모리 낭비가 걱정된다면 -> new_tensor = old_tensor.detach()
    - 메모리 낭비가 걱정된다면 -> new_tensor = old_ndarray.as_tensor()
- torch.tensor()와 Tensor.clone().detach()는 역할이 같다
- clone().detach()를 권장하는 이유는
> Tensor.clone().detach() makes it very clear what happenes to the Tensor. You first allocate new memory for it then detach it from the autograd graph from the original one.

In [ ]:
x = torch.rand(3, requires_grad=True)

print(x.requires_grad)

y = torch.tensor(x)
z = x.clone().detach()

print(y.requires_grad)
print(z.requires_grad)

True
False
False


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


### IntTensor, LongTensor, FloatTensor(=Tensor), DoubleTensor

In [ ]:
l = [[1, 2], [3, 4]]
a = np.array([[1, 2], [3, 4]])

t1 = torch.LongTensor(l)
t2 = torch.LongTensor(a)

t3 = torch.FloatTensor(l)
t4 = torch.FloatTensor(a)

print(t1, t2, t3, t4, '\n')

t1[0][0] = 10
t2[0][0] = 100
print(t1, '\n')
print(t2, '\n')
print(l, '\n')
print(a, '\n')
# list와는 메모리 공유 안하고, ndarray와는 메모리 공유하는 것인가?

tensor([[1, 2],
        [3, 4]]) tensor([[1, 2],
        [3, 4]]) tensor([[1., 2.],
        [3., 4.]]) tensor([[1., 2.],
        [3., 4.]]) 

tensor([[10,  2],
        [ 3,  4]]) 

tensor([[100,   2],
        [  3,   4]]) 

[[1, 2], [3, 4]] 

[[100   2]
 [  3   4]] 



### as_tensor
- torch.as_tensor(data, dtype, device)
- convert the data into a torch.Tensor
- share same memory
- not only numpy but list, ..
- slower than from_numpy
- https://discuss.pytorch.org/t/from-numpy-vs-as-tensor/79932https://discuss.pytorch.org/t/from-numpy-vs-as-tensor/79932
- list나 ndarray, tensor와 같은 객체를 tensor객체로 변환시켜준다
- 만약 tensor -> tensor인데 dtype, required_grad 모두 같으면 메모리 공유 -> no copy will be performed
- list, ndarray -> tensor인데 dtype 같으면 메모리 공유 -> no copy will be performed
- dtype 바뀌거나, requires_grad 바뀌면 메모리 공유 X -> copy will be performed
- (메모리 공유하는지는 a is b == True 로 확인하면 되는 것 맞나?) -> 맞다
- https://stackoverflow.com/questions/5445080/compare-if-two-variables-reference-the-same-object-in-python

In [ ]:
a = np.array([1, 2, 3])

t = torch.as_tensor(a)

t[0] = -1

print(a, t, sep='\n')
print(a is t)
print(id(a), id(b))

# t를 바꾸니까 a도 함께 바뀌는 것을 봤을 때는 메모리를 공유하는 것 같은데 memory주소는 다르다
# as_tensor는 메모리를 공유한다고 알고있는데 무엇이 문제인가

[-1  2  3]
tensor([-1,  2,  3])
False
139685700991536 139685701344624


In [ ]:
a = torch.tensor([1, 2, 3])

t = torch.as_tensor(a)

t[0] = -1

# print(a.is_leaf, t.is_leaf)

print(a is t)

True


In [ ]:
a = torch.tensor([1, 2, 3], dtype=torch.float32, requires_grad=True)

t = torch.as_tensor(a)

# print(a.is_leaf, t.is_leaf)

print(a is t)
try:
    a[0] = -1
except:
    # https://discuss.pytorch.org/t/leaf-variable-was-used-in-an-inplace-operation/308
    # requires_grad=True인 leaf variables에는 in-place operation 사용이 안된다
    print("PyTorch doesn’t allow in-place operations on leaf variables that have requires_grad=True (such as parameters of your model) \nbecause the developers could not decide how such an operation should behave. \nIf you want the operation to be differentiable, you can work around the limitation by cloning the leaf variable (or use a non-inplace version of the operator).")

True
PyTorch doesn’t allow in-place operations on leaf variables that have requires_grad=True (such as parameters of your model) 
because the developers could not decide how such an operation should behave. 
If you want the operation to be differentiable, you can work around the limitation by cloning the leaf variable (or use a non-inplace version of the operator).


In [ ]:
a = torch.tensor([1, 2, 3], dtype=torch.float32, requires_grad=True)

t = torch.as_tensor(a, dtype=torch.long)

print(a is t)

False


In [ ]:
a = np.array([1, 2, 3, 4])

b = torch.Tensor(a)

### from_numpy
- share same memory
- only numpy -> faster than as_tensor

In [ ]:
a = np.array([[1, 2], [3, 4]])

t = torch.from_numpy(a)

t[0][0] = 100
print(a)

[[100   2]
 [  3   4]]


### zeros, zeros_like, ones, ones_like
- torch.zeros(size, dtype, device, requirers_grad) (size는 sequence of intergers)
- torch.zeros_like(input, dtype, device, requires_grad)

In [ ]:
torch.zeros(2, 3)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [ ]:
x = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
y = torch.as_tensor(x)
try:
    t = torch.zeros_like(x)
except TypeError:
    print("'input' must be 'Tensor'")
    print("if you want to use ndarray as input -> torch.zeros(ndarray.shape)")
    t = torch.zeros_like(y)
    t = torch.zeros(x.shape)

t

'input' must be 'Tensor'
if you want to use ndarray as input -> torch.zeros(ndarray.shape)


tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.]])

### arange, range, linspace
- torch.arange(start, end, step, dtype, device, requires_grad)
- torch.range(start, end, step, dtype, device, requires_grad) -> 사용x
- torch.linspace(start, end, steps, dtype, device, requires_grad)

In [ ]:
print(torch.arange(5))
print(torch.arange(1, 4))
print(torch.arange(0, 2.5, 0.5))

tensor([0, 1, 2, 3, 4])
tensor([1, 2, 3])
tensor([0.0000, 0.5000, 1.0000, 1.5000, 2.0000])


In [ ]:
# 얘는 이름은 파이썬의 range와 같은데 끝을 포함한다는 점에서 다르다 -> 사용 지양 -> 파이썬 range와 유사한 torch.arange 권장
print(torch.range(1, 4))
print(torch.range(1, 4, 0.5))

tensor([1., 2., 3., 4.])
tensor([1.0000, 1.5000, 2.0000, 2.5000, 3.0000, 3.5000, 4.0000])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print(torch.linspace(3, 10, 5))
print(torch.linspace(-10, 10, 1))

tensor([ 3.0000,  4.7500,  6.5000,  8.2500, 10.0000])
tensor([-10.])


### empty, empty_like, full, full_like
- torch.empty(size, dtype, device, requires_grad, pin_memory)
    - uninitialized data -> not empty
    - dtype지정 안하면 거의 0에 가까운 값
    - size는 sequence of integers

- torch.full(size, fill_value, dtype, device, requires_grad)
    - fill_value(Scalar)로 채워진 tensor 리턴
    - size는 list, tuple,  or torch.Size

In [ ]:
a = torch.empty(2, 3, dtype=torch.int32)

# sequence of integers이지만 list, tuple같은 것도 된다.
b = torch.empty((2, 3))
c = torch.empty([2, 3])

print(a, b, c, sep='\n')

tensor([[1070479680,      21984,        114],
        [        99,        104,         46]], dtype=torch.int32)
tensor([[1.6111e+00, 3.0806e-41, 3.3631e-44],
        [0.0000e+00,        nan, 0.0000e+00]])
tensor([[1.6111e+00, 3.0806e-41, 1.5975e-43],
        [1.3873e-43, 1.4574e-43, 6.4460e-44]])


In [ ]:
torch.full((2, 3), 100)

tensor([[100, 100, 100],
        [100, 100, 100]])

### ❓ computational graph (Autograd)
https://tutorials.pytorch.kr/beginner/blitz/autograd_tutorial.html

#### Background

신경망 학습은 크게 두 단계로 나뉘어집니다.
- 순전파: forward propagation을 통해prediction을 구하고 이를 이용해 loss 계산
- 역전파: back prop을 통해 gradients를 구하고 이를 이용해 parameter들을 adjust

In [ ]:
# forward prop
# prediction = model(data)

# backward prop
# loss.backward()

#### Autograd

- 텐서에 requires_grad=True를 적용하면 이는 autograd가 이 tensor의 모든 연산들을 추적하도록 합니다
- loss가 계산되고 loss.backward()를 호출하면 autograd는 gradient를 계산하고 이를 텐서의 .grad 속성(attribute)에 저장합니다
- autograd는 실행 시점에 정의되는(define-by-run) 프레임워크입니다

#### Computation Graph (DAG)

- Autograd는 텐서의 모든 연산들의 기록을 [Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)객체로 구성된 DAG(방향성 비순환 그래프)에 저장합니다.
    - Pytorch의 computation graph를 그릴 때 각 노드에 실제로 들어가는 것은 tensor의 grad_fn
    - Function객체는 forward, backward 메서드를 갖는다
    - 이렇게 객체로 관리하는 이유는 backward 메서드에서 forward 에서의 input을 사용하므로 공유하기 위해
    - https://hwijeen.github.io/2019-01-30/Pytorch-autograd/
- chain rule을 효율적으로 계산하도록 해주는 그래프 형태
- 여기서 입력 텐서는 leaf, 출력 텐서는 root입니다.
- .backward()가 호출되면
    - 각 .grad_fn(미분함수)으로부터 gradient 계산
    - 각 텐서의 .grad 속성에 계산 결과 누적
    - chain rule이용해서 leaf까지 back propagation
- Pytorch의 DAG는 dynamic -> forward prop마다(iter) 새로운 그래프 생성
- requires_grad=False를 하면 텐서에 대한 연산 추적 그만둔다(DAG에서 gradient 계산 제외된다)

### ❓ leaf variable

- DAG 그래프에서 어떤 연산이 적용되지 않은 텐서를 leaf node라고 합니다
    - leaf node의 grad_fn은 None입니다.
- leaf node는 DAG 안에서 autograd되어야 하므로 in-place 연산이 허용되지 않습니다
- 따라서 w를 업데이트 할 때 w -= lr * w.grad로 하면 안되고 w.data -= lr * w.grad 해야합니다.
- https://m.blog.naver.com/myincizor/221691061964#

## 3. Indexing, Slicing, Joining, Mutating Ops ⭐️⭐️⭐️

### cat
- torch.cat(tensors, dim)
- concatenating되는 dim 제외한 나머지 shape은 같아야 함

In [ ]:
x1 = torch.randn(2, 3)
x2 = torch.randn(2, 1)

# 괄호로 묶어줘야함
y = torch.cat((x1, x2), dim=1)
print(y.shape)
print(x1, x2, y, sep='\n')

torch.Size([2, 4])
tensor([[-0.2995, -1.1762, -1.0677],
        [ 1.2605,  0.4792, -1.9207]])
tensor([[0.0565],
        [0.5285]])
tensor([[-0.2995, -1.1762, -1.0677,  0.0565],
        [ 1.2605,  0.4792, -1.9207,  0.5285]])


### chunk
- torch.chunk(input, chunks, dim) -> Tuple of Tensors
- chunks 값만큼 덩어리 개수가 생긴다
- 정확히 나누어지지 않으면 기대했던 결과와 다를 가능성 높다
- 사용 빈도 낮을 것 같다

In [ ]:
x = torch.randn(3, 20)

chunked_tensors = torch.chunk(x, 8, dim=1)
print(len(chunked_tensors))
print(chunked_tensors)

### stack, vstack, hstack, dstack
#### stack
- torch.stack(tensors, dim)
- concatenates a sequence of tensors along **new dimension**👈
- All tensors need to be of the **same size**👈
- dim range는 [-(Tensor.dim()+1), Tensor.dim()]이다

#### vstack
- torch.vstack(tensors)
- row방향으로 쌓는다, 세로 방향으로 쌓는다, dim=0으로 쌓는다
- (2, 3), (2, 3) -> (4, 3)
- (1, 2, 3), (1, 2, 3) -> (2, 2, 3)
- (1, 2, 3, 4), (1, 2, 3, 4) -> (2, 2, 3, 4)

#### hstack
- torch.hstack(tensors)
- col방향으로 쌓는다, 가로 방향으로 쌓는다, dim=1로 쌓는다
- (2, 3), (2, 3) -> (2, 6)
- (1, 2, 3), (1, 2, 3) -> (1, 4, 3)
- (1, 2, 3, 4), (1, 2, 3, 4) -> (1, 4, 3, 4)

#### dstack
- torch.hstack(tensors)
- depth 방향으로 쌓는다, dim=2로 쌓는다
- (2, 3), (2, 3) -> (2, 3, 2)
- (1, 2, 3), (1, 2, 3) -> (1, 2, 6)
- (1, 2, 3, 4), (1, 2, 3, 4) -> (1, 2, 6, 4)

In [ ]:
# stack
x1 = torch.randn(1, 2)
x2 = torch.randn(1, 2)

y1 = torch.stack((x1, x2), dim=0)
y2 = torch.stack((x1, x2), dim=1)
y3 = torch.stack((x1, x2), dim=2)

print(x1.shape, x2.shape, sep='\n')
print(y1.shape, y2.shape, y3.shape, sep='\n')

print(x1, x2, sep='\n')
print(y1, y2, y3, sep='\n')


torch.Size([1, 2])
torch.Size([1, 2])
torch.Size([2, 1, 2])
torch.Size([1, 2, 2])
torch.Size([1, 2, 2])
tensor([[ 0.5468, -0.3392]])
tensor([[-0.5758,  0.0322]])
tensor([[[ 0.5468, -0.3392]],

        [[-0.5758,  0.0322]]])
tensor([[[ 0.5468, -0.3392],
         [-0.5758,  0.0322]]])
tensor([[[ 0.5468, -0.5758],
         [-0.3392,  0.0322]]])


In [ ]:
# vstack
x1 = torch.randn(2, 3, 4)
x2 = torch.randn(2, 3, 4)

y = torch.vstack((x1, x2))
print(y.shape) # (4, 3, 4)
print(y)

torch.Size([4, 3, 4])
tensor([[[-0.9154, -0.2477, -0.4924,  0.2846],
         [-0.4548,  0.8675, -0.6013,  0.2208],
         [-1.6972, -3.2085, -0.7774, -0.2032]],

        [[-0.6598, -0.3618, -1.3260, -0.2642],
         [ 1.3453, -0.3754,  0.0592, -1.4695],
         [ 0.4837,  1.4451,  0.1663,  0.0582]],

        [[ 0.1270, -0.9210,  0.8833, -2.5603],
         [-0.3061, -0.2937, -1.9509, -0.8583],
         [-0.9136,  0.4834, -0.2130,  0.0528]],

        [[-0.6842,  0.9185,  0.6885,  1.5271],
         [-1.5206,  0.0565, -1.7724, -0.4526],
         [-0.3464, -0.8440, -1.1350, -1.6760]]])


In [ ]:
# hstack
x1 = torch.randn(2, 3, 4)
x2 = torch.randn(2, 3, 4)

y = torch.hstack((x1, x2))
print(y.shape) # (2, 6, 4)
print(y)

torch.Size([2, 6, 4])
tensor([[[ 1.4115e+00,  1.0316e+00, -1.1565e-01,  8.2927e-01],
         [-2.5447e-01,  3.7793e-01, -1.8413e+00, -8.9530e-01],
         [ 1.9382e-02,  1.9052e+00, -1.4014e+00, -4.8985e-02],
         [ 2.0805e-01, -9.0104e-01, -2.0825e-01,  2.2560e+00],
         [-1.0309e+00,  2.7812e-01,  1.3619e-03, -8.4614e-01],
         [ 6.3844e-01, -1.2345e+00, -1.2544e+00,  1.4346e+00]],

        [[-6.0755e-02,  7.6914e-02,  1.1443e-02,  1.0369e+00],
         [ 1.2290e+00,  8.1587e-01, -1.0058e+00, -1.5504e+00],
         [ 1.4567e+00,  4.0380e-01, -5.5423e-01,  6.4927e-02],
         [-4.9451e-01,  2.5306e-01, -1.7838e+00, -2.9622e-01],
         [ 3.1463e-01, -2.9277e-01,  1.0445e+00,  1.9620e-01],
         [ 1.8116e-01, -1.5390e-01, -5.0856e-01,  1.7285e-02]]])


In [ ]:
# dstack
x1 = torch.randn(2, 3, 4)
x2 = torch.randn(2, 3, 4)

y = torch.dstack((x1, x2))
print(y.shape) # (2, 3, 8)
print(y)

torch.Size([2, 3, 8])
tensor([[[-0.5894, -0.1969, -0.0938,  0.4447,  0.7467,  0.0866, -1.6301,
           2.2125],
         [ 1.3138, -0.9577,  0.4234,  0.9972, -0.3345,  1.8383,  0.3826,
           0.6105],
         [ 0.3751,  0.0412,  1.0531, -1.7201,  0.7172, -0.9315,  0.0231,
          -0.7532]],

        [[-0.5059,  0.4501, -1.1764, -1.2726,  0.0491,  0.0243,  0.8455,
          -1.0309],
         [-0.5638,  2.0161, -0.3661, -0.1592,  0.4538,  2.6832, -0.6364,
          -0.1315],
         [-0.1368, -2.3042,  2.0819, -1.0028, -1.6497, -0.6272,  0.5644,
          -0.1861]]])


### split, vsplit, hsplit, dsplit
#### split
- torch.split(tensor, split_size or selections, dim)
- split_size(int)인 경우 split_size 값 만큼의 size 갖는 텐서로 나눈다 (batch size같은 개념)
- selections(list)인 경우 selections의 element값 만큼을 split_size로 가진다 -> selections 원소의 합이 split하고자 하는 차원의 값과 같아야 함
- (3, 16) -> selections: [1, 5, 10] ok
- (3, 16) -> selections: [1, 5, 11] X

In [ ]:
x = torch.randn(5, 16)

y = torch.split(x, 3, dim=1)
for batch in y:
    print(batch.shape)

print('\n')

z = torch.split(x, [1, 3, 5, 7], dim=1)
for batch in z:
    print(batch.shape)

torch.Size([5, 3])
torch.Size([5, 3])
torch.Size([5, 3])
torch.Size([5, 3])
torch.Size([5, 3])
torch.Size([5, 1])


torch.Size([5, 1])
torch.Size([5, 3])
torch.Size([5, 5])
torch.Size([5, 7])


### index_select
- torch.index_select(input, dim, index)
- index를 통해 특정 index의 tensor를 가져온다
- index는 IntTensor 또는 LongTensor type의 1-D tensor이다
- input.dim()은 변하지 않는다 (3차원이면 계속 3차원)
- input의 dim dimension은 len(index)와 같아진다

In [ ]:
x = torch.randn(3, 4)

indices = torch.tensor([0, 2])

y = torch.index_select(x, dim=1, index=indices)

print(x, y, sep='\n')

tensor([[-0.5041, -1.4409,  2.1656,  2.3224],
        [-0.9715, -0.7680, -0.8635,  1.2626],
        [-0.4589,  0.5104, -0.7188, -1.4460]])
tensor([[-0.5041,  2.1656],
        [-0.9715, -0.8635],
        [-0.4589, -0.7188]])


### masked_select

### movedim, moveaxis

### nonzero

### reshape
- torch.reshape(input, shape)
- input과 data는 같다, shape만 다르다
- 가능하면 view of input 아니면 copy

In [ ]:
x = torch.arange(4.)

y = torch.reshape(x, (2, 2))

z = torch.reshape(y, (-1, ))

print(y.shape, z.shape)

print(y, z, sep='\n')

torch.Size([2, 2]) torch.Size([4])
tensor([[0., 1.],
        [2., 3.]])
tensor([0., 1., 2., 3.])


### ravel
- torch.ravel(input)
- Return a contiguous flattened tensor. A copy is made only if needed

In [6]:
x = torch.rand(2, 6)

y = torch.transpose(x, 0, 1)

z = torch.ravel(x)

print(y.is_contiguous(), y.shape)
print(z.is_contiguous(), z.shape)


False torch.Size([6, 2])
True torch.Size([12])


### squeeze, unsqueeze

### transpose
- torch.transpose(input, dim0, dim1)
- dim0과 dim1이 바뀐 tensor를 리턴
- input과 memory를 **공유**👈

In [ ]:
x = torch.randn(2, 3, 4)

y = torch.transpose(x, 0, 2) # (4, 3, 2)

print(y.shape)

y[2][1][0] = 12345

print(y[2][1][0] == x[0][1][2])

torch.Size([4, 3, 2])
tensor(True)


### swapaxes, swapdims
- transpose와 같음 -> numpy 자주 사용하는 사람들을 위해

In [ ]:
x = torch.randn(2, 3, 4)

y = torch.swapaxes(x, 0, 2) # (4, 3, 2)
z = torch.swapdims(x, 0, 2) # (4, 3, 2)

print(y.shape)
print(z.shape)

y[2][1][0] = 12345

print(y[2][1][0] == x[0][1][2] == z[2][1][0])

torch.Size([4, 3, 2])
torch.Size([4, 3, 2])
tensor(True)


### where

### gather

### ❓ view of the original tensor
- https://pytorch.org/docs/stable/tensor_view.html

### ❓ viewing vs copying

- view는 memory를 공유, copy는 memory를 공유X

### ❓ contiguous
- 연속적인 메모리
- contiguous한 텐서인지 확인: Tensor.is_contiguous()
- contiguous한 텐서 만들고 싶을 때: Tensor.contiguous()
- 텐서의 형태를 바꿀 때 인덱스는 바뀌었으나 실제 메모리 상에서는 배열을 바꾸지 않는다 -> 위치 바꾸는 연산이 잦으면 성능을 떨어트리므로
- view()같은 연산은 contiguous한 텐서만 지원하므로 Tensor.contiguous()필요
- https://titania7777.tistory.com/3
- https://discuss.pytorch.org/t/contigious-vs-non-contigious-tensor/30107

### ❓텐서 사이즈를 바꾸는 여러 가지 방법들의 차이
- torch.reshape()
    - 굉장히 유연하다
    - view될 수도, copy될 수도 있다
    - contiguous, non-contiguous 텐서 모두 지원
- Tensor.view()
    - view만 된다
    - contiguous한 텐서만 지원 (Tensor.contiguous()사용하면 error걱정 없이 view 사용가능)

- if you just want to reshape tensors, use torch.reshape.
- if you're also concerned about memory usage, use view

- Tensor.clone().view()
    - clone를 하면 memory 공유하지 않는 새로운 텐서 생성
    - memory 절대 공유하지 않으면서 사이즈 바꾸고 싶을 때는 clone 사용

- Tensor.detach().view()
    - computation graph에서 추적되지 않는 텐서를 원할 경우

## 4. Random sampling ⭐️

### seed, manual_seed, initial_seed

### bermoulli, multinomial, normal, poisson

### rand, rand_like

### randint, randint_like

### randn, randn_like

### randperm

## 5.Serialization (Save & Load) ⭐️

### save, load

## 6. Parallelism ( parallelism on CPU)

### get_num_threads

### set_num_threads

## 7. Locally disabling gradient computation

### no_grad

### enable_grad

### set_grad_enabled

### is_grad_enabled

### inference_mode

### is_inference_mode_enabled

## 8. Math operations
- Pointwise Ops
- Reduction Ops
- Comparison Ops
- Spectral Ops
- Other Ops

## Pointwise Ops ⭐️

### add, sub, mul, div, remainder

### ceil, floor, round

### clip, trunc

### cos, sin, sign, sigmoid

### pow, sqrt, rsqrt

### imag, real, conj, abs

### gradient

### log, exp

## Reduction Ops ⭐️



### max, min

### argmax, argmin

### all, any

### mean, median, sum, prod

### mode

### norm

### unique

### count_nonzero

## Comparison Ops ⭐️

### argsort, kthvalue, srt, topk, msort

### eq, ge, gt

### isclose, isfinite, isinf, isnan, isreal

### maximum, minimum

### sort, argsort, ,sork

### kthvalue, topk

## Spectral Ops

### stft, istft

### hamming_window, hann_window, kaiser_window

## Other Ops

### clone
- torch.clone(input)
- input의 copy 리턴

In [ ]:
x = torch.rand(2, 3, requires_grad=True)

y = torch.clone(x)

z = torch.detach(x)

print(x, y, z, sep='\n')

print(y.requires_grad)
print(z.requires_grad)

tensor([[0.1392, 0.3353, 0.9397],
        [0.8237, 0.1135, 0.7326]], requires_grad=True)
tensor([[0.1392, 0.3353, 0.9397],
        [0.8237, 0.1135, 0.7326]], grad_fn=<CloneBackward>)
tensor([[0.1392, 0.3353, 0.9397],
        [0.8237, 0.1135, 0.7326]])
True
False


### cumprod, cumsum

### diag, diagonal

### trace

### diff

### einsum

### flatten

### flip

### combinations

## BLAS and LAPACK Ops

### addbmm, addmm, bmm

### matmul, mm

### dot, inner

### eig

### inverse

### lstsq